<a href="https://colab.research.google.com/github/Meha223/Bank-Marketing/blob/main/Bank_Marketing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install duckdb

In [2]:
from google.colab import files
uploaded = files.upload()

Saving bank.csv to bank.csv


In [3]:
import pandas as pd
df = pd.read_csv('bank.csv')
df.head()

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,deposit
0,59,admin.,married,secondary,no,2343,yes,no,unknown,5,may,1042,1,-1,0,unknown,yes
1,56,admin.,married,secondary,no,45,no,no,unknown,5,may,1467,1,-1,0,unknown,yes
2,41,technician,married,secondary,no,1270,yes,no,unknown,5,may,1389,1,-1,0,unknown,yes
3,55,services,married,secondary,no,2476,yes,no,unknown,5,may,579,1,-1,0,unknown,yes
4,54,admin.,married,tertiary,no,184,no,no,unknown,5,may,673,2,-1,0,unknown,yes


In [4]:
# Connect duckdb
import duckdb

con = duckdb.connect()

In [5]:
# Register dataset

con.register('bank.csv', df)

In [12]:
# Create base table with a surrogate key

con.execute("""
    CREATE OR REPLACE TABLE bank_base AS
    SELECT
        ROW_NUMBER() OVER () AS fact_id,
        *
    FROM bank.csv
""")

In [13]:
# Preview client table
con.execute("SELECT * FROM bank_base LIMIT 5").df()

,fact_id,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,deposit
0,1,59,admin.,married,secondary,False,2343,True,False,unknown,5,may,1042,1,-1,0,unknown,True
1,2,56,admin.,married,secondary,False,45,False,False,unknown,5,may,1467,1,-1,0,unknown,True
2,3,41,technician,married,secondary,False,1270,True,False,unknown,5,may,1389,1,-1,0,unknown,True
3,4,55,services,married,secondary,False,2476,True,False,unknown,5,may,579,1,-1,0,unknown,True
4,5,54,admin.,married,tertiary,False,184,False,False,unknown,5,may,673,2,-1,0,unknown,True


In [14]:
# Create normalized tables

# Client table

con.execute("""
    CREATE OR REPLACE TABLE client AS
    SELECT DISTINCT
      fact_id,
      ROW_NUMBER() OVER() AS client_id, age, job, marital, education, "default" AS credit_default, housing, loan
    FROM bank_base
""")

# Add age group in client table

con.execute("""
    ALTER TABLE client ADD COLUMN age_group VARCHAR(255);
    UPDATE client
    SET age_group = CASE
      WHEN age<25 THEN '18-24'
      WHEN age>24 AND age<35 THEN '25-34'
      WHEN age>35 AND age<45 THEN '35-44'
      WHEN age>45 AND age<55 THEN '45-54'
      WHEN age>=55 THEN '55 and above'
      END;
""")

In [15]:
# Preview client table
con.execute("SELECT * FROM client LIMIT 5").df()

,fact_id,client_id,age,job,marital,education,credit_default,housing,loan,age_group
0,2,2,56,admin.,married,secondary,False,False,False,55 and above
1,3,3,41,technician,married,secondary,False,True,False,35-44
2,10,10,28,services,single,secondary,False,True,False,25-34
3,11,11,38,admin.,single,secondary,False,True,False,35-44
4,12,12,30,blue-collar,married,secondary,False,True,False,25-34


In [16]:
# Campaign table

con.execute("""
    CREATE OR REPLACE TABLE campaign AS
    SELECT DISTINCT fact_id, ROW_NUMBER() OVER() AS campaign_id, campaign, pdays, previous, poutcome
    FROM bank_base
""")

In [17]:
# Preview campaign table
con.execute("SELECT * FROM campaign LIMIT 5").df()

,fact_id,campaign_id,campaign,pdays,previous,poutcome
0,2,2,1,-1,0,unknown
1,3,3,1,-1,0,unknown
2,5,5,2,-1,0,unknown
3,9,9,1,-1,0,unknown
4,11,11,1,-1,0,unknown


In [18]:
# Contact table

con.execute("""
    CREATE OR REPLACE TABLE contact AS
    SELECT DISTINCT fact_id, ROW_NUMBER() OVER() AS contact_id, contact, month, day, duration
    FROM bank_base
""")

# Add duration length in Contact table

con.execute("""
    ALTER TABLE contact ADD COLUMN duration_length VARCHAR(255);
    UPDATE contact
    SET duration_length =
    CASE
        WHEN duration > 0 AND duration <= 60 THEN 'Short'
        WHEN duration > 60 AND duration <= 300 THEN 'Medium'
        ELSE 'Long'
    END;
""")

In [19]:
# Preview Contact table
con.execute("SELECT * FROM contact LIMIT 5").df()

,fact_id,contact_id,contact,month,day,duration,duration_length
0,1,1,unknown,may,5,1042,Long
1,2,2,unknown,may,5,1467,Long
2,3,3,unknown,may,5,1389,Long
3,5,5,unknown,may,5,673,Long
4,6,6,unknown,may,5,562,Long


In [20]:
# Create marketing table

con.execute("""
    CREATE OR REPLACE TABLE marketing AS
    SELECT
        b.fact_id,
        c.client_id,
        ct.contact_id,
        ca.campaign_id,
        b.deposit AS subscribed
    FROM bank_base b
    JOIN client c ON b.fact_id = c.fact_id
    JOIN contact ct ON b.fact_id = ct.fact_id
    JOIN campaign ca ON b.fact_id = ca.fact_id
""")

In [21]:
# Preview marketing table
con.execute("SELECT * FROM marketing LIMIT 5").df()

,fact_id,client_id,contact_id,campaign_id,subscribed
0,1,1,1,1,True
1,2,2,2,2,True
2,3,3,3,3,True
3,4,4,4,4,True
4,5,5,5,5,True


In [22]:
# Export to Parquet

con.execute("COPY client TO 'client.parquet' (FORMAT 'parquet')")
con.execute("COPY contact TO 'contact.parquet' (FORMAT 'parquet')")
con.execute("COPY campaign TO 'campaign.parquet' (FORMAT 'parquet')")
con.execute("COPY marketing TO 'marketing.parquet' (FORMAT 'parquet')")

In [23]:
# Generate MermaidJS Erd

import os
import zipfile

# 1. Create folder if it doesn't exist
folder_path = "/mnt/bankmarketing"
os.makedirs(folder_path, exist_ok=True)

# 2. Save Mermaid ERD to a .mmd file
mermaid_erd = """
erDiagram
    bank_base {
        int fact_id PK
        int age
        string job
        string marital
        string education
        string default
        string housing
        string loan
        string contact
        string month
        int day
        int duration
        int campaign
        int pdays
        int previous
        string poutcome
        string deposit
    }

    client {
        int client_id PK
        int fact_id FK
        int age
        string job
        string marital
        string education
        string credit_default
        string housing
        string loan
    }

    contact {
        int contact_id PK
        int fact_id FK
        string contact
        string month
        int day
        int duration
    }

    campaign {
        int campaign_id PK
        int fact_id FK
        int campaign
        int pdays
        int previous
        string poutcome
    }

    fact_campaign_results {
        int fact_id PK
        int client_id FK
        int contact_id FK
        int campaign_id FK
        string subscribed
    }

    bank_base ||..|| client : "fact_id"
    bank_base ||..|| contact : "fact_id"
    bank_base ||..|| campaign : "fact_id"
    fact_campaign_results ||--|| client : "client_id"
    fact_campaign_results ||--|| contact : "contact_id"
    fact_campaign_results ||--|| campaign : "campaign_id"
"""

erd_path = os.path.join(folder_path, "bank_marketing_erd.mmd")
with open(erd_path, "w") as f:
    f.write(mermaid_erd)

# 3. Create a zip file containing the folder
zip_path = "/mnt/bankmarketing.zip"
with zipfile.ZipFile(zip_path, 'w', zipfile.ZIP_DEFLATED) as zipf:
    for root, _, files in os.walk(folder_path):
        for file in files:
            file_path = os.path.join(root, file)
            zipf.write(file_path, os.path.relpath(file_path, start=folder_path))

# 4. Display download link
from google.colab import files
files.download(zip_path)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [24]:
# Schema documentation

import os

# Define the directory to save the README.md file
readme_path = "/mnt/bankmarketing/README.md"
os.makedirs(os.path.dirname(readme_path), exist_ok=True)

# Define the schema documentation content
schema_doc = """
# Bank Marketing Project - Database Schema

## Tables Overview

### 1. **bank_base**
The base table containing all the data before splitting into dimension and fact tables. It includes raw information from the `bank.csv`.

| Column        | Data Type | Description                                                |
|---------------|-----------|------------------------------------------------------------|
| fact_id       | INT       | Surrogate key, unique for each row                         |
| age           | INT       | Age of the customer                                        |
| job           | STRING    | Job type of the customer                                   |
| marital       | STRING    | Marital status                                             |
| education     | STRING    | Education level                                            |
| credit_default| STRING    | Whether the customer has credit_default                    |
| housing       | STRING    | Housing status                                             |
| loan          | STRING    | Loan status                                                |
| contact       | STRING    | Contact information                                        |
| month         | STRING    | Month of the contact                                       |
| day           | INT       | Day of the contact                                         |
| duration      | INT       | Duration of the contact                                    |
| campaign      | INT       | Campaign information                                       |
| pdays         | INT       | Number of days passed after the last contact               |
| previous      | INT       | Number of contacts performed before this one               |
| poutcome      | STRING    | Outcome of the previous marketing campaign                  |
| deposit       | STRING    | Whether the customer subscribed to the deposit product     |

### 2. **client**
Client dimension table which includes client-specific attributes.

| Column           | Data Type | Description                                              |
|------------------|-----------|----------------------------------------------------------|
| client_id        | INT       | Surrogate key for the client                             |
| fact_id          | INT       | Foreign key, references `bank_base.fact_id`              |
| age              | INT       | Age of the client                                        |
| job              | STRING    | Job type of the client                                   |
| marital          | STRING    | Marital status                                           |
| education        | STRING    | Education level                                          |
| credit_default   | STRING    | Whether the client has credit_default                    |
| housing          | STRING    | Housing status                                           |
| loan             | STRING    | Loan status                                              |

### 3. **contact**
Contact dimension table with information about contact details.

| Column        | Data Type | Description                                              |
|---------------|-----------|----------------------------------------------------------|
| contact_id    | INT       | Surrogate key for the contact                            |
| fact_id       | INT       | Foreign key, references `bank_base.fact_id`              |
| contact       | STRING    | Contact information                                      |
| month         | STRING    | Month of the contact                                     |
| day           | INT       | Day of the contact                                       |
| duration      | INT       | Duration of the contact                                  |

### 4. **campaign**
Campaign dimension table with marketing campaign-related data.

| Column        | Data Type | Description                                              |
|---------------|-----------|----------------------------------------------------------|
| campaign_id   | INT       | Surrogate key for the campaign                           |
| fact_id       | INT       | Foreign key, references `bank_base.fact_id`              |
| campaign      | INT       | Campaign identifier                                      |
| pdays         | INT       | Number of days passed after the last contact             |
| previous      | INT       | Number of previous campaigns                             |
| poutcome      | STRING    | Outcome of the previous campaign                         |

### 5. **marketing**
Fact table that consolidates data from dimension tables and indicates subscription status.

| Column          | Data Type | Description                                              |
|-----------------|-----------|----------------------------------------------------------|
| fact_id         | INT       | Surrogate key, references `bank_base.fact_id`            |
| client_id       | INT       | Foreign key, references `client.client_id`               |
| contact_id      | INT       | Foreign key, references `contact.contact_id`             |
| campaign_id     | INT       | Foreign key, references `campaign.campaign_id`           |
| subscribed      | STRING    | Indicates if the customer subscribed to the deposit      |

# Synthetic Features

- age_group in client table: Range of ages
- duration_length in contact table: Categorizes duration into qualitative labels.

## Relationships

- **bank_base** table serves as the base for all other tables, providing a unique `fact_id` for each row.
- The **client**, **contact**, and **campaign** tables are dimension tables that link to the **marketing** table.
- The **marketing** table stores the relationships between clients, their contact details, and the campaigns they are associated with.

"""

# Save the schema documentation to README.md
with open(readme_path, "w") as file:
    file.write(schema_doc)

# Display message indicating file is saved
print(f"Schema documentation saved to {readme_path}")

Schema documentation saved to /mnt/bankmarketing/README.md
